In [2]:
import pandas as pd
import sqlite3 as db
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler, OrdinalEncoder, FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
# python
import pickle
from functools import partial
# basics
import scipy.stats as stats
# graphing
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.metrics import plot_confusion_matrix
# feature selection
from sklearn.feature_selection import SelectKBest, SelectPercentile, chi2
# model selection
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score
from sklearn.metrics import (r2_score, mean_squared_error, accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, roc_curve, precision_recall_curve, make_scorer,
                             confusion_matrix, multilabel_confusion_matrix, ConfusionMatrixDisplay)
# models
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LogisticRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.dummy import DummyClassifier
import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor
from imblearn.over_sampling import SMOTE
from collections import Counter
from datetime import datetime
from functions import training_dataset, PrepareWordLists, selectdata, readdata, cleandatastructure, buildwordsindex, trainingstructuredata, getlistofwords
#import importlib as implib
import gc

import unidecode
import string
import re
import json
from names_dataset import NameDataset

## Import Excel File



In [4]:
#ISM_SourceData = pd.read_excel('Data/Tickets 2021-2022 (V02).xlsx', sheet_name="Squirrel SQL Export")
#ISM_SourceData.info()
ISM_SourceData = readdata()

In [3]:
#ISM_SourceData.info()
#with open('ISM_SourceData.dataframe', 'wb') as f:
#    pickle.dump(ISM_SourceData, f, protocol=pickle.HIGHEST_PROTOCOL)
with open('ISM_SourceData.dataframe', 'rb') as f:
    ISM_SourceData = pickle.load(f)

In [4]:
ISM_SourceData_test = ISM_SourceData[( ISM_SourceData['ACTUALSTART']>="2021-07-25") & (ISM_SourceData['ACTUALSTART']<="2021-08-03")]
ISM_SourceData_test.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 9585 entries, 5 to 427445
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   TICKETID                   9585 non-null   object        
 1   TITLE                      9585 non-null   object        
 2   LONGDESCRIPTION            9542 non-null   object        
 3   STATUS                     9585 non-null   object        
 4   IMPACT                     9585 non-null   float64       
 5   URGENCY                    9585 non-null   float64       
 6   AFFECTEDPERSON             9585 non-null   object        
 7   AffectedPersonName         9580 non-null   object        
 8   AffectedPersonEmail        9019 non-null   object        
 9   AffectedPersonCountry      9585 non-null   object        
 10  CREATIONDATE               9585 non-null   datetime64[ns]
 11  CREATEDBY                  9585 non-null   object        
 12  Crea

## Cleaning Data

In [5]:
country_list = ['Mexico']

#ISM_test = ISM_test[ISM_test['AffectedPersonCountry'] in country_list == ]
ISM_test = cleandatastructure(ISM_SourceData_test,country_list )
#ISM_test = ISM_test[ISM_test['AffectedPersonCountry'].isin(country_list)]
ISM_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4956 entries, 17 to 427439
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   TICKETID               4956 non-null   object
 1   FULLDESC               4956 non-null   object
 2   AffectedPersonCountry  4956 non-null   object
 3   OWNERGROUP             4956 non-null   object
dtypes: object(4)
memory usage: 193.6+ KB


C:\Users\mamoralessr\Documents\GitHubRepo\MarioMorales-CEMEX-FinalProject\functions.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ISM_SourceData_test['FULLDESC'] = ISM_SourceData_test['TITLE']+ ' ' +ISM_SourceData_test['LONGDESCRIPTION']
C:\Users\mamoralessr\Anaconda3\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
C:\Users\mamoralessr\Documents\GitHubRepo\MarioMorales-CEMEX-FinalProject\functions.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

## 

In [8]:
df_InputData = pd.DataFrame(columns=['TICKETID','FULLDESC','AffectedPersonCountry','OWNERGROUP'])
df_InputData.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   TICKETID               0 non-null      object
 1   FULLDESC               0 non-null      object
 2   AffectedPersonCountry  0 non-null      object
 3   OWNERGROUP             0 non-null      object
dtypes: object(4)
memory usage: 0.0+ bytes


In [3]:
#with open('ISM_test.dataframe', 'wb') as f:
#    pickle.dump(ISM_test, f, protocol=pickle.HIGHEST_PROTOCOL)
with open('ISM_test.dataframe', 'rb') as f:
    ISM_test = pickle.load(f)
ISM_test['AffectedPersonCountry'].count()

14450

## Prepare Gorups Word Lists


In [10]:
## Split test data into sections for performance issue
#dataset_size = 2000


In [91]:
ISM_test_set1=ISM_test[0:4956]
ISM_test_set2=ISM_test[5000:9999]
ISM_test_set3=ISM_test[10000:14449]
#ISM_test_set4=ISM_test[30000:39999]
#ISM_test_set5=ISM_test[40000:44608]
#ISM_test_set6=ISM_test[25000:29000]
#ISM_test_set6=ISM_test[10000:10999]
#ISM_test_set7=ISM_test[12000:12999]
#ISM_test_set8=ISM_test[14000:15999]
#ISM_test_set9=ISM_test[16000:17356]

In [92]:
## Deffine the data set to processes
datasets1_2_3 = [ISM_test_set1,ISM_test_set2,ISM_test_set3]
datasets1 = [ISM_test_set1]
datasets2 = [ISM_test_set2]
datasets3 = [ISM_test_set3]
datasets4 = [ISM_test_set4]
datasets5 = [ISM_test_set5]

In [10]:
## Crear estructura para guardar los datos
ISM_wgwords = pd.DataFrame(columns=['COUNTRY', 'OWNERGROUP', 'WORDS'])

## Run de classfier words programs
## Regresa lista de palabras para cada país y groupo

#ISM_wgwords0 = PrepareWordLists(datasets1_2_3_4, ISM_wgwords)
#ISM_wgwords1 = PrepareWordLists(datasets2, ISM_wgwords0)
#ISM_wgwords2 = PrepareWordLists(datasets3, ISM_wgwords1)
ISM_wgwords4 = PrepareWordLists(datasets4, ISM_wgwords)

## Consolidate word lists if required
#ISM_wgwords_all = ISM_wgwords0

#ISM_wgwords_all = ISM_wgwords0.append(ISM_wgwords1)
#ISM_wgwords_all = ISM_wgwords_all.append(ISM_wgwords2)

## Save file on excel to analyze
#ISM_wgwords_all.to_excel(f"./Data/ISMwgwords2.xlsx")

#with open('ISM_wgwords_all', 'wb') as f:
#    pickle.dump(ISM_wgwords_all, f, protocol=pickle.HIGHEST_PROTOCOL)
#with open('ISM_wgwords_all', 'rb') as f:
#    ISM_wgwords_all = pickle.load(f)


2022-03-29 18:20:34.678456
1
2022-03-29 18:20:37.570698
Words:  1373  Groups:  31  Countries: 1


In [11]:
ISM_wgwords_all = ISM_wgwords4

In [14]:
## Generate a unique datasat with Index for faster search

ISM_wgwords_uniques = buildwordsindex(ISM_wgwords_all, 2)

#with open('ISM_wgwords_uniques_longprocess', 'wb') as f:
#    pickle.dump(ISM_wgwords_uniques, f, protocol=pickle.HIGHEST_PROTOCOL)
#with open('ISM_wgwords_uniques_longprocess', 'rb') as f:
#    ISM_wgwords_uniques = pickle.load(f)

#ISM_wgwords_uniques = ISM_wgwords_uniques[ISM_wgwords_uniques['COUNTRY']=='Mexico']
ISM_wgwords_uniques
ISM_wgwords_uniques.to_excel(f"./Data/ISM_wgwords_uniques_longprocess.xlsx")

## Preparara Datos

In [13]:
## Preparar archivo para training
## Country / <categoríaspor grupos con el número de palabras coincidentes> / Grupo <target>


In [8]:
#GroupColumnsNames =ISM_wgwords_uniques['OWNERGROUP'].unique()
#TrainingData = pd.DataFrame(columns=GroupColumnsNames)
#TrainingData['COUNTRY'] = 'NA'
#TrainingData['OWNERGROUP'] = 'NA'
#TrainingData['TICKETID'] = 'NA'

#TrainingData.info()
#TrainingData.COUNTRY.datatype = object
#TrainingData.OWNERGROUP.datatype = object
#TrainingData.TICKETID.datatype = object
#for column in TrainingData.columns:
#    print(column)

TrainingData = trainingstructuredata(ISM_wgwords_uniques)
TrainingData

,I-CEX-IN-ECS-O365-L3,I-CEX-MX-IT-SEC-DTC-L2,I-CEX-IN-IT-IAM-DTC-L2,I-CEX-MX-COM-LE-GSC-L1,I-CEX-MXBOSAP-SEC-GRC-L3,I-CEX-US-IBMSECURITYUSA-L3,I-CEX-EMEAA-BOCOM-GSC-L1,I-CEX-GLOBAL-IT-CNX-L2,I-CEX-GLOBAL-IT-LNT-GSC-L1,I-CEX-GLOBAL-IT-CNX-MON-L2,...,I-CEX-US-DSS-WPB-L2,I-CEX-MX-HR-SER-PR-L2,I-CEX-MX-CONTACTCENTER-GV-L2,C-CEX-MX-CONTACTCENTER-GV-L2,I-CEX-GLOBAL-IT-MSSSOC-INTQRAD-DTC-L2,I-CEX-MX-IT-SEC-CAB,I-CEX-MX-IT-SCCM-APP-L2,COUNTRY,OWNERGROUP,TICKETID


In [111]:
ISM_Training_1

,I-CEX-MXBOSAP-SEC-APP-L3,I-CEX-MX-OP-GINCO-GSC-L1,I-CEX-MX-COM-SAP-SL-APP-L3,I-CEX-MX-COM-LE-GSC-L1,I-CEX-IN-IT-IAM-DTC-L2,I-CEX-EMEAA-BOCOM-SEC-L1,I-CEX-MX-BO-JDE-GSC-L1,I-CEX-GLOBAL-ITBDCIAMAP-GSC-L1,I-CEX-LATAM-AFST-CO-PR-MNT-L3,I-CEX-GLOBAL-IT-DYN-L2,...,I-CEX-CZ-IT-GSC-L1,I-CEX-GLOBAL-IT-MSSSOC-INTQRAD-DTC-L2,C-CEX-AE-BO-GL-SEC-L2,I-CEX-MX-IT-SEC-CAB,C-CEX-PA-IT-INM-ROS-OCC-L2,I-CEX-MX-CONTACTCENTER-GV-L2,I-CEX-MX-IT-DISPR-DTC-L2,COUNTRY,OWNERGROUP,TICKETID
0,484,576,625,729,529,100,841,625,676,400,...,1,0,0,0,1,1,0,Mexico,I-CEX-MX-PSD-LE-PA-L3,SR2535931
1,4,9,4,9,9,0,16,16,4,1,...,0,0,0,0,1,0,0,Mexico,I-CEX-NOAM-IT-GSC-L1,SR2530945
2,9,16,1,16,16,0,25,25,1,4,...,0,0,0,0,1,0,0,Mexico,I-CEX-NOAM-IT-GSC-L1,SR2531002
3,529,1681,400,1296,1369,64,1225,961,289,484,...,1,0,0,4,1,0,1,Mexico,I-CEX-GLOBAL-IT-ATT-L3,SR2529754
4,36,36,36,36,36,16,36,36,36,36,...,1,0,1,1,0,0,0,Mexico,I-CEX-GLOBAL-IT-LNT-GSC-L1,SR2529871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4951,16,16,4,25,16,0,36,36,1,4,...,0,0,0,0,1,0,0,Mexico,I-CEX-NOAM-IT-GSC-L1,SR2534586
4952,25,64,81,169,64,4,64,49,64,49,...,1,0,0,0,0,0,0,Mexico,I-CEX-MX-COM-LE-GSC-L1,SR2534636
4953,484,625,676,841,900,36,961,900,256,529,...,1,0,0,0,0,0,0,Mexico,I-CEX-MX-IT-SER-DEPOT-L2,SR2534095
4954,4,9,4,9,9,0,16,16,4,1,...,0,0,0,0,1,0,0,Mexico,I-CEX-NOAM-IT-GSC-L1,SR2534827


In [9]:
#ISM_Training_1 = training_dataset(ISM_test_set1, TrainingData, ISM_wgwords_uniques)
square_values = True

ISM_Training_1 = training_dataset(ISM_test_set1, TrainingData, ISM_wgwords_uniques, square_values)

2022-03-28 21:08:06.904739
1
2022-03-28 21:08:07.196798
2
2022-03-28 21:08:07.712137
4
2022-03-28 21:08:08.717430
8
2022-03-28 21:08:10.713872
16
2022-03-28 21:08:14.876066
32
2022-03-28 21:08:23.404342
64
2022-03-28 21:08:39.915647
128
2022-03-28 21:09:12.776344
256
2022-03-28 21:10:20.234540
512
2022-03-28 21:12:32.898482
1024
2022-03-28 21:16:56.748936
2048
2022-03-28 21:25:53.602908
4096
Finished:  2022-03-28 21:29:58.812505  Errors: 804839


In [10]:
ISM_Training_2 = training_dataset(ISM_test_set2, TrainingData, ISM_wgwords_uniques, square_values)

2022-03-28 21:29:58.834859
1
2022-03-28 21:29:59.115974
2
2022-03-28 21:29:59.675366
4
2022-03-28 21:30:00.703161
8
2022-03-28 21:30:02.790291
16
2022-03-28 21:30:07.111518
32
2022-03-28 21:30:15.707047
64
2022-03-28 21:30:33.158878
128
2022-03-28 21:31:08.328222
256
2022-03-28 21:32:15.495609
512
2022-03-28 21:34:38.435469
1024
2022-03-28 21:39:15.918560
2048
2022-03-28 21:48:28.886601
4096
Finished:  2022-03-28 21:52:31.359345  Errors: 804839


In [11]:
ISM_Training_3 = training_dataset(ISM_test_set3, TrainingData, ISM_wgwords_uniques, square_values)

2022-03-28 21:53:52.936637
1
2022-03-28 21:53:53.185020
2
2022-03-28 21:53:53.720313
4
2022-03-28 21:53:54.806177
8
2022-03-28 21:53:56.992196
16
2022-03-28 21:54:01.817094
32
2022-03-28 21:54:10.484366
64
2022-03-28 21:54:28.406810
128
2022-03-28 21:55:02.498226
256
2022-03-28 21:56:10.391463
512
2022-03-28 21:58:23.583856
1024
2022-03-28 22:02:55.737471
2048
2022-03-28 22:12:03.288364
4096
Finished:  2022-03-28 22:13:38.247701  Errors: 716289


In [14]:
#with open('ISM_Training.dataframe_1', 'wb') as f:
#    pickle.dump(ISM_Training, f, protocol=pickle.HIGHEST_PROTOCOL)
#with open('ISM_Training.dataframe_1', 'rb') as f:
#    ISM_Training_load = pickle.load(f)


In [95]:
#Integrar set de training_data y ajustar el indice al TICKETID
ISM_Training_load = ISM_Training_1
#ISM_Training_load = ISM_Training_1.append(ISM_Training_2)
#ISM_Training_load = ISM_Training_load.append(ISM_Training_3)
#ISM_Training_load = ISM_Training_load.append(ISM_Training_4)

In [109]:
#with open('ISM_Training_load.dataframe', 'wb') as f:
#    pickle.dump(ISM_Training_load, f, protocol=pickle.HIGHEST_PROTOCOL)
#with open('ISM_Training_load.dataframe', 'rb') as f:
#    ISM_Training_load = pickle.load(f)

ISM_Training_load

,I-CEX-MXBOSAP-SEC-APP-L3,I-CEX-MX-OP-GINCO-GSC-L1,I-CEX-MX-COM-SAP-SL-APP-L3,I-CEX-MX-COM-LE-GSC-L1,I-CEX-IN-IT-IAM-DTC-L2,I-CEX-EMEAA-BOCOM-SEC-L1,I-CEX-MX-BO-JDE-GSC-L1,I-CEX-GLOBAL-ITBDCIAMAP-GSC-L1,I-CEX-LATAM-AFST-CO-PR-MNT-L3,I-CEX-GLOBAL-IT-DYN-L2,...,I-CEX-CZ-IT-GSC-L1,I-CEX-GLOBAL-IT-MSSSOC-INTQRAD-DTC-L2,C-CEX-AE-BO-GL-SEC-L2,I-CEX-MX-IT-SEC-CAB,C-CEX-PA-IT-INM-ROS-OCC-L2,I-CEX-MX-CONTACTCENTER-GV-L2,I-CEX-MX-IT-DISPR-DTC-L2,COUNTRY,OWNERGROUP,TICKETID
0,484,576,625,729,529,100,841,625,676,400,...,1,0,0,0,1,1,0,Mexico,I-CEX-MX-PSD-LE-PA-L3,SR2535931
1,4,9,4,9,9,0,16,16,4,1,...,0,0,0,0,1,0,0,Mexico,I-CEX-NOAM-IT-GSC-L1,SR2530945
2,9,16,1,16,16,0,25,25,1,4,...,0,0,0,0,1,0,0,Mexico,I-CEX-NOAM-IT-GSC-L1,SR2531002
3,529,1681,400,1296,1369,64,1225,961,289,484,...,1,0,0,4,1,0,1,Mexico,I-CEX-GLOBAL-IT-ATT-L3,SR2529754
4,36,36,36,36,36,16,36,36,36,36,...,1,0,1,1,0,0,0,Mexico,I-CEX-GLOBAL-IT-LNT-GSC-L1,SR2529871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4951,16,16,4,25,16,0,36,36,1,4,...,0,0,0,0,1,0,0,Mexico,I-CEX-NOAM-IT-GSC-L1,SR2534586
4952,25,64,81,169,64,4,64,49,64,49,...,1,0,0,0,0,0,0,Mexico,I-CEX-MX-COM-LE-GSC-L1,SR2534636
4953,484,625,676,841,900,36,961,900,256,529,...,1,0,0,0,0,0,0,Mexico,I-CEX-MX-IT-SER-DEPOT-L2,SR2534095
4954,4,9,4,9,9,0,16,16,4,1,...,0,0,0,0,1,0,0,Mexico,I-CEX-NOAM-IT-GSC-L1,SR2534827


In [46]:
with open('ISM_Training_load.dataframe', 'wb') as f:
    pickle.dump(ISM_Training_load, f, protocol=pickle.HIGHEST_PROTOCOL)

## Start Pipeline

In [19]:
## Create Y Data / Transform Y Data


## Clase Prepare Data
#    Instance Object()
#    Fit(ISM_Trainind_Data)
#    X_train, X_test, y_train, y_test = Transform()




In [97]:
##Convertimos los grupos en clasificaicones numéricas

ownergroup_mapping = pd.DataFrame(ISM_Training_load['OWNERGROUP'].unique(), columns=['OWNERGROUP'])
ownergroup_mapping['MAP_VALUE'] = ownergroup_mapping.index
ownergroup_mapping.head(3)

,OWNERGROUP,MAP_VALUE
0,I-CEX-MX-PSD-LE-PA-L3,0
1,I-CEX-NOAM-IT-GSC-L1,1
2,I-CEX-GLOBAL-IT-ATT-L3,2


In [98]:
#ISM_Training_load['INDEX']=ISM_Training_load.index
ISM_Training_load_full = pd.merge(ISM_Training_load, ownergroup_mapping, on='OWNERGROUP')
ISM_Training_load_full.head(3)

## Construimos X y Y

X_Train_Data = ISM_Training_load_full.drop(['OWNERGROUP', 'MAP_VALUE'], axis=1)
Y_Train_Data = ISM_Training_load_full[['TICKETID','MAP_VALUE']]
Y_Train_Data = Y_Train_Data.set_index('TICKETID') 

Y_Train_Data.head(5)

##Split Trainng Data 
## Tenemos que hacer el encoder antes, por el tema de los países, pueden salir columnas diferentes
## Al productizar, tenemos que creear una matriz de países y que sa la estructura base e ignorar el resto.

datatrans_encoder = ColumnTransformer([
    ('one_hot', OneHotEncoder(handle_unknown='ignore'), ['COUNTRY'])
], remainder='passthrough', verbose_feature_names_out=False)

df_temp = datatrans_encoder.fit_transform(X_Train_Data)
df_temp = pd.DataFrame(df_temp)
df_temp.columns = datatrans_encoder.get_feature_names_out()

X_train_Data_fixed = df_temp
X_train_Data_fixed = X_train_Data_fixed.set_index('TICKETID') 
X_train_Data_fixed.head(5)

## Hacemos Split de Datos

X_train, X_test, y_train, y_test = train_test_split(X_train_Data_fixed, Y_Train_Data, test_size=0.33)

## Cambio tipo de datos a númerico para procesos de training de modelos
X_train_fixed = X_train.apply(pd.to_numeric)
X_test_fixed = X_test.apply(pd.to_numeric)


X_train_fixed.head(5)

,COUNTRY_Mexico,I-CEX-MXBOSAP-SEC-APP-L3,I-CEX-MX-OP-GINCO-GSC-L1,I-CEX-MX-COM-SAP-SL-APP-L3,I-CEX-MX-COM-LE-GSC-L1,I-CEX-IN-IT-IAM-DTC-L2,I-CEX-EMEAA-BOCOM-SEC-L1,I-CEX-MX-BO-JDE-GSC-L1,I-CEX-GLOBAL-ITBDCIAMAP-GSC-L1,I-CEX-LATAM-AFST-CO-PR-MNT-L3,...,C-CEX-GLOBAL-ACTIVETRAC-L2,I-CEX-ES-BO-AP-APP-L2,C-CEX-CO-IT-INM-ROS-OCC-L2,I-CEX-CZ-IT-GSC-L1,I-CEX-GLOBAL-IT-MSSSOC-INTQRAD-DTC-L2,C-CEX-AE-BO-GL-SEC-L2,I-CEX-MX-IT-SEC-CAB,C-CEX-PA-IT-INM-ROS-OCC-L2,I-CEX-MX-CONTACTCENTER-GV-L2,I-CEX-MX-IT-DISPR-DTC-L2
TICKETID,,,,,,,,,,,,,,,,,,,,,
SR2537899,1.0,100,169,144,196,121,1,169,169,144,...,1,0,0,0,0,0,0,0,0,0
SR2536922,1.0,25,64,49,64,36,1,49,49,16,...,1,0,0,0,0,0,1,0,0,0
SR2537696,1.0,100,361,484,625,169,49,289,256,196,...,1,4,1,1,0,0,0,0,0,0
SR2539951,1.0,81,49,49,100,169,64,100,169,36,...,0,0,0,0,1,0,0,0,0,0
SR2541967,1.0,1,1,4,4,4,1,4,4,1,...,0,0,0,0,0,0,0,0,0,0


## Base Models

In [99]:
### Dummy Model
dum_model = DummyClassifier(strategy='uniform').fit(X_train_fixed, y_train)

In [100]:
yhat_dc = dum_model.predict(X_train_fixed)
accuracy_score(y_train, yhat_dc), precision_score(y_train, yhat_dc, average='macro'), recall_score(y_train, yhat_dc, average='macro')

(0.007530120481927711, 0.008039477157559325, 0.0068897046786538805)

In [ ]:
## Baseline
## Objective: Reduce at least 50% time of agetn assignment time
## Average assignment rate 3 min / Ticket = 10 tickets = 30 mins
##                            50%  Ticket = 5 * 1 + 5 * 3 = 22.5 = 66% (of current used time)
##                            60%  Ticket = 6 * 1 + 4 * 3 = 21.5 = 60%
##                            70%  Ticket = 7 * 1 + 3 * 3 = 19.5 = 53%
##                            80%  Ticket = 8 * 1 + 2 * 3 = 18.0 = 46%
##                            90%  Ticket = 9 * 1 + 1 * 3 = 16.5 = 40%


In [101]:
predictmodel = xgb.XGBClassifier()
predictmodel.fit(X_train_fixed,y_train)

C:\Users\mamoralessr\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = 

[16:56:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [102]:
y_train_predicted = predictmodel.predict(X_train_fixed)

In [39]:
#y_train_predicted

In [103]:
## (0.988050784167289, 0.8999303353339181, 0.9141716566866268)
## (0.9917849141150112, 0.9316936463693539, 0.9420673076923076) - 3 dataset para pabalar 1 daata set de training
## (0.9940253920836445, 0.9401629426474146, 0.9503105590062112) - 3 dataset para pabalar + optimización de palabras , 1 daata set de training 
#                                                                 / optimización performaance training data
## (0.9977595220313666, 0.9498606141920094, 0.958139534883721) - 3 dataset para palabras y training - mejora de palabras
## (1.0, 1.0, 1.0)   - solo datos de méxico, la distribucóón de palabras incorrecta
## (1.0, 1.0, 1.0)   - solo datos de méxico (1-set), la distribucóón de palabras sigue incorrecta
## (0.9970104633781763, 0.9652777777777778, 0.9722222222222222) - Solo 1000 registros mexico, mejora depuración regex, eliminar repeticiones  2
## (0.9970104633781763, 0.9670220326936746, 0.9701492537313433) - Solo 1000 registros mexico, mejora depuración regex, eliminar repeticiones  2 - eliminar nombres
## (0.9996292176492398, 0.999947254958034, 0.9998651979956652) - Todo Mexico (8056) -  mejora depuración regex, eliminar repeticiones  2 - eliminar nombres
## (0.996292176492399, 0.9893615163073012, 0.9871562449500731) - Anterior + el cuadrado de match de palabras
## (0.9994438264738599, 0.9797980294920234, 0.9841933139534884) - El proceso anteiorr + curadro de los valores sobre la tabal de trainings
## (0.9948341770844096, 0.9977036850547525, 0.9924358040679491) - Agreagar un mes completo de datos para training
## (0.8462651100320281, 0.946159775684978, 0.8879776222642395) - Con la optimización de los proceosos 
## (0.9948341770844096, 0.9894551664890539, 0.9879969490353675) - Con optimización de procesos arreglada.
## (0.9925612150015497, 0.9789320377926272, 0.9764036881612383) - Con optimización de procesos arreglada + ajustes de reducción de palabras
## (0.992974480834797, 0.9882678776778713, 0.9822549605857234) - Con 3 meses de listas de palabras, 1 mes de training, refiniación de exclusiones y eliminar palabras menos de 3 veces repetidas
## (0.9940076454179151, 0.9967231762161324, 0.98945677220278) - Con 3 meses de listas de palabras, 1 mes de training, refiniación de exclusiones y eliminar palabras menos de 2 veces repetidas

##(0.9939759036144579, 0.9955638654323862, 0.9940332624020907) - Prueba con set de datos de otra fecha
accuracy_score(y_train, y_train_predicted), precision_score(y_train, y_train_predicted, average='macro'), recall_score(y_train, y_train_predicted, average='macro')

(0.9939759036144579, 0.9955638654323862, 0.9940332624020907)

In [104]:
y_test_predicted = predictmodel.predict(X_test_fixed)

In [105]:
## (0.7333333333333333, 0.4699628058733169, 0.45581237966684646)
## (0.6348484848484849, 0.3386554337862025, 0.3552215795132643) 3 dataset para pabalar 1 daata set de training
## (0.6166666666666667, 0.31582311366794125, 0.33113032812469756) 3 dataset para pabalar + optimización de palabras , 1 daata set de training 
#                                                                  / optimización performaance training data
## (0.7434343434343434, 0.509846497720536, 0.4785910693035512)  3 dataset para palabras y training - mejora de palabras
## (0.7603461249059443, 0.48218784214388655, 0.42512328682302225) - solo datos de méxico, la distribucóón de palabras incorrecta
## (0.7348484848484849, 0.446154673658955, 0.4402168010724709) - solo datos de méxico 1-set, la distribucóón de palabras incorrecta
## (0.7045454545454546, 0.3638892520212744, 0.3594193305460011) - solo datos de méxico 1-set, la distribucóón de palabras icorregida
## (0.7242424242424242, 0.4070510067010857, 0.3896439704587102) - Solo 1000 registros mexico, mejora depuración regex, eliminar repeticiones  2
## (0.6787878787878788, 0.36347548907295174, 0.37880837133970213)- Solo 1000 registros mexico, mejora depuración regex, eliminar repeticiones  2 - eliminar nombres
## (0.7994732881866065, 0.5708462677095836, 0.49161033884617494) - Todo Mexico (8056) -  mejora depuración regex, eliminar repeticiones  2 - eliminar nombres
## (0.7968397291196389, 0.5515521425067268, 0.48502100207879784) - MIsmo que anterior.. (porque hay diferencia??)
## (0.7990970654627539, 0.5824423370084997, 0.5295225892375374) - Anterior + el cuadrado de match de palabras
## (0.8116610738255033, 0.6460636633552914, 0.5525619906042211) - Magreagar un mes completo de datos para training - solo cuadrado de math de palabras
## (0.8265520134228188, 0.6105236353198223, 0.5303955669707777) - Mismo que anterior pero otro día.
## (0.6260486577181208, 0.3651391577365122, 0.30792491216925527) - Con la optimización del Procesos
## (0.8148070469798657, 0.5686148528852358, 0.4990927061925084) - Con la optimización de Procesos arreglada
## (0.8296979865771812, 0.6506835711097666, 0.5732821390857165) - Con optimización de procesos arreglada + ajustes de reducción de palabras
## (0.7835570469798657, 0.5428910030787344, 0.46989271037760044) - Con 3 meses de listas de palabras, 1 mes de training, refiniación de exclusiones y eliminar palabras menos de 3 veces repetidas
## (0.7867030201342282, 0.5122388352939511, 0.4421900442388091) - Con 3 meses de listas de palabras, 1 mes de training, refiniación de exclusiones y eliminar palabras menos de 2 veces repetidas

## (0.6925427872860636, 0.4096398651702086, 0.36482851848552333) - Trainin Oct - nov / Test Cierre Julio-Ago
accuracy_score(y_test, y_test_predicted), precision_score(y_test, y_test_predicted, average='macro'), recall_score(y_test, y_test_predicted, average='macro')


C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.6925427872860636, 0.4096398651702086, 0.36482851848552333)

## Grid Search

In [106]:

gs = GridSearchCV(predictmodel, {'n_estimators':range (1, 10, 100), 'max_depth':range (1, 50, 100)}, cv=5, n_jobs=-1, verbose=0, scoring = 'accuracy')
gs.fit(X_train_fixed, y_train)
gs.best_score_ , gs.best_params_

C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


KeyboardInterrupt: 

In [84]:
predictmodel_final = xgb.XGBClassifier({'max_depth': 1, 'n_estimators': 1})
predictmodel_final.fit(X_train_fixed,y_train)

C:\Users\mamoralessr\AppData\Roaming\Python\Python39\site-packages\xgboost\core.py:499: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  warnings.warn(


[12:42:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [144]:
with open('XGBoostmodel', 'wb') as f:
    pickle.dump(predictmodel_final, f, protocol=pickle.HIGHEST_PROTOCOL)

In [59]:
parameters = {'n_estimators': range(100, 200, 25),
              'criterion': ['gini', 'entropy'],
              'max_depth': list(range(1, 5)) + [None],
              'max_features': ['auto', 'sqrt', 'log2']
             }
model = RandomForestClassifier()
gs = GridSearchCV(model, parameters, cv=5, n_jobs=-1, verbose=1)
gs.fit(X_train_fixed, y_train)
gs.best_score_, gs.best_params_

Fitting 5 folds for each of 120 candidates, totalling 600 fits


C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:910: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:426: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


(0.720941230486685,
 {'criterion': 'gini',
  'max_depth': None,
  'max_features': 'auto',
  'n_estimators': 150})

In [85]:
Forestmodel = RandomForestClassifier(**gs.best_params_)
Forestmodel.fit(X_train_fixed, y_train)



C:\Users\MAMORA~1\AppData\Local\Temp/ipykernel_6732/4239747531.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  Forestmodel.fit(X_train_fixed, y_train)


RandomForestClassifier(max_depth=1, n_estimators=1)

In [107]:
y_test_predicted = Forestmodel.predict(X_test_fixed)
accuracy_score(y_test, y_test_predicted), precision_score(y_test, y_test_predicted, average='macro'), recall_score(y_test, y_test_predicted, average='macro')

C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.008557457212713936, 0.0002079002079002079, 0.009615384615384616)

In [147]:
with open('Forestmodel', 'wb') as f:
    pickle.dump(Forestmodel, f, protocol=pickle.HIGHEST_PROTOCOL)

## Final Model

In [108]:
##1- (0.7333333333333333, 0.4699628058733169, 0.45581237966684646)
##2- (0.6348484848484849, 0.3386554337862025, 0.3552215795132643)
##3- (0.6166666666666667, 0.31582311366794125, 0.33113032812469756)
##4- (0.7434343434343434, 0.509846497720536, 0.4785910693035512) - 3 dataset para palabras y training - mejora de palabras
##5- (0.7603461249059443, 0.48218784214388655, 0.42512328682302225) - solo datos de méxico, la distribucóón de palabras incorrecta
##6- (0.7348484848484849, 0.446154673658955, 0.4402168010724709) - solo datos de méxico 1-set, la distribucóón de palabras incorrecta
##7- (0.7045454545454546, 0.3638892520212744, 0.3594193305460011) - solo datos de méxico 1-set, la distribucóón de palabras icorregida
##8- (0.7242424242424242, 0.4070510067010857, 0.3896439704587102) - Solo 1000 registros mexico, mejora depuración regex, eliminar repeticiones  2
##9- (0.6787878787878788, 0.36347548907295174, 0.37880837133970213)- Solo 1000 registros mexico, mejora depuración regex, eliminar repeticiones  2 - eliminar nombres
##10-(0.7994732881866065, 0.5708462677095836, 0.49161033884617494) - Todo Mexico (8056) -  mejora depuración regex, eliminar repeticiones  2 - eliminar nombres
##   (0.7968397291196389, 0.5515521425067268, 0.48502100207879784) - MIsmo que anterior.. (porque hay diferencia??) - Solo se hizo limpieza de código y agregaar index
##11-(0.7990970654627539, 0.5824423370084997, 0.5295225892375374) - Mismo que anterior + cuadrado de match de palabras.. accuracy ligeramente mejor y mejora principalment en precision y recall
##11A (0.7990970654627539, 0.5419410058175115, 0.5031069709866498) - Se utilizó el cuadrado sobre la tabla de training directamente, resultó mejor el resultado de cuadrados sobre la match de palabras.
##12 (0.8116610738255033, 0.6460636633552914, 0.5525619906042211) - Magreagar un mes completo de datos para training - solo cuadrado de math de palabras
##13 (0.9058305369127517, 0.8843810833150799, 0.7534218420171158) - Al utilzar el random forest en lugar de XGBoost obtuvimos un valor mejor.

##14 (0.8265520134228188, 0.6105236353198223, 0.5303955669707777) - XGBoost Mismo que el 12, corrido otro día ¿¿??
##15 (0.7514681208053692, 0.5208306844606043, 0.35885690027673145) - Mismo que el 13 (Random Forest), corrido otro día ¿¿??

## 16(0.6017197986577181, 0.3493015666918488, 0.2566750765676476) - Proceso Performance Optimizado - Forrest Model - "Errro en optimización"
## 17(0.6260486577181208, 0.3651391577365122, 0.30792491216925527) - Proceso Performance Optimizado - Classifier - "Errro en optimización"

## 18(0.7430788590604027, 0.5026517897869982, 0.363345094482876) -Random Forest - Proceso Optimizado y Ajustado
## 19((0.8148070469798657, 0.5686148528852358, 0.4990927061925084) - Xgboost Classifies - Proceso Optimizado y Ajustado

## 20 ((0.764261744966443, 0.4698835524311577, 0.3497842624086199) - -Random Forest - Proceso Optimizado y Ajustado + ajuste en reducción de palabras
## 21 (0.8296979865771812, 0.6506835711097666, 0.5732821390857165) - XGBoost  Proceso Optimizado y Ajustado + ajuste en reducción de palabras + 1.5% Best Model

## 22 (0.7275587248322147, 0.48831557617325067, 0.33139467071146755) - Random Forest . Resultados Similares, un poco menor
## 23 (0.7835570469798657, 0.5428910030787344, 0.46989271037760044) - Nos baja de ochenta.

## 24 (0.20092281879194632, 0.002721509350615658, 0.013109598730619969) - Random Forest
## 25 (0.7867030201342282, 0.5122388352939511, 0.4421900442388091) - Se mantiene muy cerca al anterion, quitando solo los de 1 repetición

##Test
## (0.6925427872860636, 0.4096398651702086, 0.36482851848552333) Cierre Jul-Aug - Baja el % de eficience 10%
y_test_predicted = predictmodel.predict(X_test_fixed)
accuracy_score(y_test, y_test_predicted), precision_score(y_test, y_test_predicted, average='macro'), recall_score(y_test, y_test_predicted, average='macro')



C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.6925427872860636, 0.4096398651702086, 0.36482851848552333)

In [139]:
## Conclusiones
## Los parametros proporcioandos por el grid search no nosmejoran la predicción,
## Debemos buscarle en la seleccción de las palabras

## Entrenar con el mismo set con el que se generan los set de palabras mejora sustancialmente el resultado
## Reducir el alcance a México y mejorar el filtrado de palabras debe subir al menos un 10%

## Corridas 6 y 7
## Al parecer el usar un solo set (2000 registros) si impacta negativamente.. adicionalmente la limpieza de palabras parece tener un impacto negativo en ese punto
## Siguietne Acció: Utilizar el set completo de México / y Analizar el set de palabras y exlusiones

## Corrida 9
## Eliminar los nombres en el modelo a 1000 registros resultó negativo
## Corrida 10
## Confirmamos mejor resultado.. El número de registros de de training sumado a la eliminación de nombres impacta positivamente en la predicción
##   Siguiente paso: 
##      Ajustar el proceso con el index = ticketid para poder analyzar los registros fallidos
##      Analizar los registros fallidos, existe alguna relación entre estos grupos?
##           Revisando los casos fallidos, vemos el el valor lógico si conside con el valor Real llevando el % de efectividad a 95%
##           Habilitar un exponenciador del resultod (valores cuadrados por ejemplo) debería ayudar a las predicciones
## Corrida 11
##   utilizando ls cuadrados del match de palabras, se mejora principalmente el recall y precision.. 
##      ? Hipotesis: Si aplicamos el cuadrado a la suma total, mejorará el resultado?
##      Incrementar el número de registros para set de trainnings (2 meses)
##      Darle peso excesivo a las palabras que hacen match con el group name        
## Corrida 11A
##   Se elevaron al cuadrado los valors de la tabal de training anterior, cayeron los valores resultó mejor el resultado de cuadrados sobre la match de palabras.
## Corrida 12
##  Se entrenó con un mes completo de información, logramo incremento de .012% de mejor.
##  --Aún falta idnetificar que modelo podría considerar mejor los dataos que las fallas, que en la matoría e valor mayor es el correcto.
##  --Podmeos hacer tambien un mo delo que evalúe directamente sobre el valor mayor.
## Corrida 13
##  -- Utilizando el modelo RandomForest se obtiene in 90%, las caracteristicas de este Modelo favorecen a la estructura de nuestra información
## Corrida 14 y 15: porque variaron los valores del día anterior?
## Corrida 16 y 17: El proceso optimizado debe tener algun probla en las salidas de los valores
##                    Porque los resultados son muy distintos?
##                    Existía error en el proceso optimizado
## Corrida 18 y 19:
##  18 - 
##  19 - 
##  20 - Random Forest: da un máximo de 75%
##  21 - xgboostClassifier: da un máximo de .82%  Mejor model  (corrección de erroers de exclusion list)
##
##  Últimas acciones:
##    Hacer una revalidación de las palabras de exclusión basado en el listofwords unique optimizado - done
##    Crecer el Set de Palabras   - done
##    Escoger un set de datos basado en los períodos com mas tickets del año. incluir cierre de año
##    Escoger un set de datos muestra de otros períodos 
##    Hacer el ejercicio de Entrenamiento y prédicción.
##
## Nuevas Conclusiones, antes de quitar palabras y dejar convinaciones únicas, debemos darle peso a cada una según su nivel de repetición, y de las que se encutren que hagan match
##    sacar la suma de los pesos y utilizar eso como número a asignar.. (y puede ser al cuadrado)
##    Porqué el modelo coo archivo no dá el mismo resultado???


In [122]:
## Preapramos información para revisión y analisis de errores.

ownergroup_mapping
groups_dic = {ownergroup_mapping.loc[idx, 'MAP_VALUE']: ownergroup_mapping.loc[idx, 'OWNERGROUP'] for idx in range(ownergroup_mapping.shape[0])}
X_test_fixed['PREDICT'] = y_test_predicted
X_test_fixed['REAL'] = y_test
X_test_fixed['PREDICT'] = X_test_fixed['PREDICT'].map(groups_dic)
X_test_fixed['REAL'] = X_test_fixed['REAL'].map(groups_dic)
X_test_fixed[['PREDICT','REAL']].head(3)




,PREDICT,REAL
TICKETID,,
SR2651035,I-CEX-MX-COM-LE-GSC-L1,I-CEX-MX-OP-SAP-RDM-APP-L3
SR2670267,I-CEX-MX-OP-SAP-PROC-L3,I-CEX-MX-OP-SAP-PROC-L3
SR2674081,I-CEX-NOAM-IT-GSC-L1,I-CEX-NOAM-IT-GSC-L1


In [123]:
X_test_fixed.to_excel(f"./Data/ResultAnalysis_Forest.xlsx")



In [1]:
#X_test_fixed

## Test NameDataset

In [12]:
nd = NameDataset()

In [15]:
dict_names = nd.search('##CUCARAMACARATITIRIFUE##').get('first_name')


In [16]:
print(dict_names)

None


In [25]:
#print(nd.search('STEVEN').get('first_name').get('country').get('Mexico'))
errcount = 0
try:
    is_name = nd.search('MARIO').get('first_name').get('country').get('Mexico')
    if is_name == 'None':
        print('graba')
except:
    errcount = errcount + 1
    print('graba1')
#else:
#    errcount = errcount + 1
        
#print(nd.search('CUCARAMACARATITIRIFUE').get('first_name').get('country').get('Mexico'))

In [99]:
type(x)

NoneType

In [103]:
if nd.search('mistico').get('first_name') == x :
    print('None-1')
else:
    print(nd.search('TELEFONO'))



{'first_name': None, 'last_name': None}


## Test 1

In [50]:
count = 0
count2 = 0
count_check = 1


for numindex, ind in enumerate(ISM_test.index):
    count = count + 1
    if count == count_check:
        print(datetime.now())
        print(count)
        
        if count == count_check:
            count_check = count_check + count_check
        
    cleanstring = ISM_test['FULLDESC'][ind].replace("\\N"," ")
    cleanstring = cleanstring.replace(r"\N"," ")
    word_list = cleanstring.split(" ")
    
    
    #TrainingData = TrainingData.append({'COUNTRY':ISM_test['AffectedPersonCountry'][ind],'OWNERGROUP':ISM_test['OWNERGROUP'][ind]}, index=[ind]) 
    #ISM_wgwords = ISM_wgwords.append({'COUNTRY':ISM_test['AffectedPersonCountry'][ind],'OWNERGROUP':ISM_test['OWNERGROUP'][ind],'WORDS':word.upper()}, ignore_index=True)
    
    TrainingData = TrainingData.append([ISM_test['AffectedPersonCountry'][ind],ISM_test['OWNERGROUP'][ind]], ['COUNTRY','OWNERGROUP'],[numindex])
    #TrainingData = TrainingData.append({'COUNTRY':ISM_test['AffectedPersonCountry'][ind],'OWNERGROUP':ISM_test['OWNERGROUP'][ind]}, ignore_index=True) 
    
    for column in TrainingData.columns:
        if column not in ['0','COUNTRY','OWNERGROUP']:
            TrainingData[column][ind] = 0
            
    
    #print(ISM_test['AffectedPersonCountry'][ind])
    #print(TrainingData['COUNTRY'][ind])
    
    listwords = ISM_wgwords_uniques[(ISM_wgwords_uniques['COUNTRY']==ISM_test['AffectedPersonCountry'][ind])][['WORDS','OWNERGROUP']]
    #print(len(listwords))
    
    for word in word_list:
        if len(word) > 2:
                if word.upper() not in ['[Screenshot','THE','AND','ATTACHMENT','PLEASE','FOR','CEMEX','SUBJECT','FAVOR','RE:','CAUTION','REGARDS','SALUDOS','listener.]','FROM:','SUBJECT:','CC:','TO:','FW:',\
                                         'SENT:','GSC', 'THIS', 'DEL','POR','PARA','QUE','CON'] \
                and word.find('cid:') == -1 and word.find('cid:') == -1  and word.find('@cemex.com') == -1 and word.find('@ext.cemex.com') == -1 and word.find('image') == -1 and word.find('>>;') == -1 \
                and word.find('GSC') == -1 and word.find('______') == -1 and word.find('---') == -1 and word.find('  ') == -1 and word.find('.gif') == -1 and word.find('safelinks') == -1 \
                and word.find('.png') == -1 and word.find('E0') == -1 and word.find('E-') == -1 and word.find('****') == -1 and word.find('#') == -1 and word.find('listener.]') and word.find('you') and word.find('Regards') \
                and word.find('del') and word.find('para') and word.find('por') and word.find('saved') and word.find('was') and word.find('are') and word.find('del') \
                and word.find('e-mail') and word.find('este') and word.find('[Screenshot') and word.find('las') and word.find('los') and word.find('not') and word.find('Thank')\
                and not word.startswith('+') and not word[0].isdigit() and not word[1].isdigit() and not word.startswith('<') and not word.startswith('/>') and not word.startswith('@'):
                    
                    #grouplist = ISM_wgwords_uniques[ISM_wgwords_uniques['COUNTRY']==ISM_test['AffectedPersonCountry'][ind]][['OWNERGROUP']]
                    #for group in grouplist['OWNERGROUP']:
                    #    print(group)
                    #    #listwords = ISM_wgwords_uniques[ISM_wgwords_uniques['OWNERGROUP']==group & ISM_wgwords_uniques['COUNTRY']==ISM_test['AffectedPersonCountry'][ind]][['WORDS']]
                    #    listwords = ISM_wgwords_uniques[(ISM_wgwords_uniques['OWNERGROUP']==group)&(ISM_wgwords_uniques['COUNTRY']==ISM_test['AffectedPersonCountry'][ind])][['WORDS']]
                    #    if word in listwords['WORDS']:
                    #        print(group)
                    #        TrainingData[group][ind] = TrainingData[group][ind] + 1
                    #print('Valid Word')
                    #print(word)
                                        
                    #for idx1 in listwords:
                    #    print(idx1)
                        #print(listwords['WORDS'][idx1])
                    #if word == listwords['WORDS'][idx1]:
                    #print(group)
                    #    TrainingData[listwords['OWNERGROUP']][ind] = TrainingData[listwords['OWNERGROUP']][ind] + 1
                    
                    for listidx in listwords.index:
                        if listwords['WORDS'][listidx] == word :
                            #print(listwords['OWNERGROUP'][listidx])
                            #print("Word Found")
                            #print('TrainingData[',listwords['OWNERGROUP'][listidx],'][',ind,']','= TrainingData[',listwords['OWNERGROUP'][listidx],'][',ind,'] + 1')
                            #print(TrainingData[listwords['OWNERGROUP'][listidx]][ind])
                            #print(numindex)
                            TrainingData[listwords['OWNERGROUP'][listidx]][numindex] = TrainingData[listwords['OWNERGROUP'][listidx]][numindex] + 1
                            #print(TrainingData[listwords['OWNERGROUP'][listidx]][ind])

TrainingData.info()


2022-03-21 13:43:52.479470
1
2022-03-21 13:43:52.695937
2
2022-03-21 13:43:59.035910
4
2022-03-21 13:44:05.338098
8
2022-03-21 13:44:21.304474
16
2022-03-21 13:44:39.254132
32
2022-03-21 13:45:41.697980
64
2022-03-21 13:46:31.151217
128
2022-03-21 13:49:52.843760
256


KeyboardInterrupt: 

In [542]:
TrainingData.describe()
TrainingData[['COUNTRY','OWNERGROUP','I-CEX-MX-OP-GINCO-GSC-L1','I-CEX-NOAM-IT-GSC-L1']]

,COUNTRY,OWNERGROUP,I-CEX-MX-OP-GINCO-GSC-L1,I-CEX-NOAM-IT-GSC-L1
0,Colombia,I-CEX-MX-OP-GINCO-GSC-L1,10,4
1,GB,C-CEX-FR-BSO-FA-PR-L2,0,0
2,India,I-CEX-EMEA-IT-SERVICE DESK-L1,0,0
3,Mexico,I-CEX-NOAM-IT-GSC-L1,3,5
4,Mexico,I-CEX-NOAM-IT-GSC-L1,5,11
5,Spain,I-CEX-ES-IT-INM-ROS-VLC-L2,0,4
6,Colombia,I-CEX-NOAM-IT-GSC-L1,30,158


## Prueba SET

In [114]:
#cleanstring = ISM_test['FULLDESC'][1].replace("\\N"," ")
#cleanstring = cleanstring.replace(r"\N"," ")
#word_list = cleanstring.split(" ")
#word_list



In [115]:
#grouplistwords = ISM_wgwords_uniques[(ISM_wgwords_uniques['COUNTRY']==ISM_test['AffectedPersonCountry'][1]) &(ISM_wgwords_uniques['OWNERGROUP']==ISM_test['OWNERGROUP'][1]) ][['WORDS']]
#xxlist = grouplistwords['WORDS'].to_list()
#yylist = set(word_list) & set(xxlist)
#yylist

In [28]:

GroupColumnsNames =ISM_wgwords['OWNERGROUP'].unique()
TrainingData = pd.DataFrame(columns=GroupColumnsNames)
TrainingData['COUNTRY'] = 'NA'
TrainingData['OWNERGROUP'] = 'NA'

TrainingData.info()
TrainingData.COUNTRY.datatype = object
TrainingData.OWNERGROUP.datatype = object

count = 0
count2 = 0
count_check = 1


for numindex, ind in enumerate(ISM_test.index):
    count = count + 1
    if count == count_check:
        print(datetime.now())
        print(count)
        
        if count == count_check:
            count_check = count_check + count_check
            #if count_check == 32:
            #    break
            
    cleanstring = ISM_test['FULLDESC'][ind].replace("\\N"," ")
    cleanstring = cleanstring.replace(r"\N"," ")
    word_list = cleanstring.split(" ")
    
    TrainingData = TrainingData.append({'COUNTRY':ISM_test['AffectedPersonCountry'][ind],'OWNERGROUP':ISM_test['OWNERGROUP'][ind]}, ignore_index=True) 
    #TrainingData = TrainingData.append([ISM_test['AffectedPersonCountry'][ind],ISM_test['OWNERGROUP'][ind]], ['COUNTRY','OWNERGROUP'],[numindex])
    
    ## Seteo valores de columnas de grupos del registros nuevo para que no sean null y puedan sumarse
    for column in TrainingData.columns:
        if column not in [0,'COUNTRY','OWNERGROUP']:
            TrainingData[column][numindex] = 0
    
    for columns in GroupColumnsNames:
        grouplistwords = ISM_wgwords_uniques[(ISM_wgwords_uniques['COUNTRY']==ISM_test['AffectedPersonCountry'][ind]) &(ISM_wgwords_uniques['OWNERGROUP']==columns) ][['WORDS']]
        grouplistwords_list = grouplistwords['WORDS'].to_list()
        
        ## el & de lista busca obtiene lso valores que hacen match
        list_matches = set(word_list) & set(grouplistwords_list)
        if len(list_matches) > 0:
            TrainingData[columns][numindex] = TrainingData[columns][numindex] + len(list_matches)


TrainingData.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Columns: 183 entries, I-CEX-MX-BO-JDE-GSC-L1 to OWNERGROUP
dtypes: object(183)
memory usage: 0.0+ bytes
2022-03-21 15:53:48.799973
1
2022-03-21 15:53:49.619435
2
2022-03-21 15:53:51.418975
4


KeyboardInterrupt: 

In [30]:
TrainingData

,I-CEX-MX-BO-JDE-GSC-L1,I-CEX-MX-OP-GINCO-GSC-L1,I-CEX-MXBOSAP-SEC-APP-L3,I-CEX-MX-COM-LE-GSC-L1,I-CEX-NOAM-IT-GSC-L1,I-CEX-MX-PSD-LE-PA-L3,I-CEX-GLOBAL-IT-CNX-L2,I-CEX-GLOBAL-IT-LNT-GSC-L1,I-CEX-MX-IT-SER-RS-L2,I-CEX-ES-IT-INM-ROS-VLC-L2,...,I-CEX-US-COMMS-L3,C-CEX-MX-IT-GINCOSOL-APP-L2,I-CEX-IN-IT-MAILCOL-DTC-L2,I-CEX-MX-HR-SER-PR-L2,C-CEX-GLOBAL-IT-AXIANS-DTC-L2,V-CEX-AT&T-WAN,C-CEX-HR-IT-CP-L2,I-CEX-US-APPS-S&OP,COUNTRY,OWNERGROUP
0,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,United Kingdom,C-CEX-FR-BSO-FA-PR-L2
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Jamaica,I-CEX-US-SERVICEDESK-L1
2,23,12,14,17,18,9,4,16,4,0,...,0,1,0,0,0,0,0,0,Mexico,C-CEX-DE-IT-SAP-SEC-L2
3,7,0,0,4,5,0,0,0,0,5,...,0,0,0,0,0,0,0,0,Spain,I-CEX-NOAM-IT-GSC-L1
4,18,17,11,17,17,12,9,15,6,0,...,0,1,0,2,0,0,0,0,Mexico,I-CEX-MXBOSAP-PRCL3
5,16,24,13,41,14,21,14,27,29,0,...,0,0,0,1,0,0,0,0,Mexico,I-CEX-MX-COM-SAP-SL-APP-L3
6,2,2,0,6,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,Mexico,I-CEX-MX-COM-LE-GSC-L1


## Word CLeaning testing

In [79]:
import unidecode
import string
import re


wdlist = ['uno', 'UNO', '  UNÓ', 'Uno<<', 'unó:[]', 'uno: ']
wdlist = [re.sub('[?:"><\'. ()!¡\[\]]', '',unidecode.unidecode(wd).upper()) for wd in wdlist]
#new_string = a_string.translate(str.maketrans('', '', string.punctuation))
wdlist = list(dict.fromkeys(wdlist))
wdlist

['UNO']

In [65]:
for wd in wdlist: 
    print()

uno
UNO
UNÓ
Uno
unó:
uno:


In [73]:
import re

word = "  esto esto:?  "
word = re.sub('[?:"><\'.()!¡]', '',word)
word

'  esto esto  '

In [ ]:
# import required module
import unidecode
 
# assign string
string = "orčpžsíáýd"
 
# display original string
print('\nOriginal String:', string)
 
# remove ascents
outputString = unidecode.unidecode(string)
 
# display new string
print('\nNew String:', outputString)

## Performance Optimizaction Test

In [30]:
def PrepareWordLists_1(datasets, df_words):
    # Recibe la lista de los datasets seccionados, para evitar tiempos muy largos de procesamiento 
    nd = NameDataset()
    no_name = nd.search('##CUCARAMACARATITIRIFUE##').get('first_name')
    for dataset in datasets:
        errcount = 0
        count = 0
        count_check = 1
        #print(dataset.info())

        for ind in dataset.index:
            #Seperar las palabras de cada regitro
            #Crear un registro para cada palabra
            count = count + 1
            if count == count_check:
                print(datetime.now())
                print(count)
                count_check = count_check + 500

            word_list = getlistofwords_1(dataset['FULLDESC'][ind], nd)
            word_list = list(dict.fromkeys(word_list))
            
            for word in word_list:  
                df_words = df_words.append({'COUNTRY':dataset['AffectedPersonCountry'][ind],'OWNERGROUP':dataset['OWNERGROUP'][ind],'WORDS':word.upper()}, ignore_index=True)
 
                            
    print(datetime.now())
    print('Words: ',len(df_words['WORDS'].unique())," Groups: ",len(df_words['OWNERGROUP'].unique()),' Countries:',len(df_words['COUNTRY'].unique()))
    return df_words

def cleandatastructure_1(ISM_SourceData_test, country_list):
    ## Put all text fields together
    ISM_SourceData_test['FULLDESC'] = ISM_SourceData_test['TITLE']+ ' ' +ISM_SourceData_test['LONGDESCRIPTION']
    ISM_SourceData_test['FULLDESC']
    ISM_test = ISM_SourceData_test[['TICKETID','FULLDESC','AffectedPersonCountry','OWNERGROUP']]

    ## Do some Cleaning
    ISM_test['FULLDESC'].fillna('NA',inplace = True)
    ISM_test['AffectedPersonCountry'].fillna('NA',inplace = True)
    ISM_test['FULLDESC'] = ISM_test['FULLDESC'].str.upper()
    ISM_test['FULLDESC'] = ISM_test['FULLDESC'].str.replace("\\N"," ")
    ISM_test['FULLDESC'] = ISM_test['FULLDESC'].str.replace(r"\N"," ")
    #ISM_test['FULLDESC'] = re.sub('<.*@.*>', ' ',ISM_test['FULLDESC'])
    #ISM_test['FULLDESC'] = re.sub('[-_"&.,\[\]?!¡#<>;:\'=$()*\\\]', ' ',cleanstring)
    
    #Delete Null values
    ISM_test = ISM_test[ISM_test['AffectedPersonCountry']!="NA"]
    ISM_test = ISM_test[ISM_test['FULLDESC']!="NA"]

    ## Consolidate Countriees
    ISM_test['AffectedPersonCountry'] = ISM_test['AffectedPersonCountry'].map({'DOM': 'Dominican Republic','Bahamas': 'Bahamas', 'Bangladesh': 'Banglasdesh',\
                                        'Belize': 'Belize','Central': 'Central', \
                                        'China': 'China','Clombia': 'Colombia','Colombia': 'Colombia','Costa Rica': 'Costa Rica','Croatia': 'Croatia','CZ': 'Czech Republic',\
                                        'Czech Republic': 'Czech Republic','DE': 'Germany','Dominican Republic': 'Dominican Republic', \
                                        'EG': 'Egypt','EGY': 'Egypt','Egypt': 'Egypt','El Salvador': 'El Salvador','ES': 'Spain','España': 'Spain',\
                                        'Filipinas': 'Philippines','FR': 'France','France': 'France', \
                                        'GB': 'United Kingdom','German': 'Germany','Germany': 'Germany','Great Britain': 'United Kingdom','Guatemala': 'Guatemala',\
                                        'Filipinas': 'Philippines',\
                                        'Haiti': 'Haiti','HU': 'Hungary','IL': 'Israel','Israel': 'Israel','Jamaica': 'Jamaica','JM': 'Jamaica','Malaysia': 'Malaysia',\
                                        'Latvia': 'Latvia','Mexic': 'Mexico',\
                                        'México': 'Mexico','Mexio': 'Mexico','Mexixo': 'Mexico','Mexico': 'Mexico','Filipinas': 'Philippines','MX': 'Mexico',\
                                        'Filipinas': 'Philippines','Netherlands': 'Netherlands',\
                                        'NIC': 'Nicaragua','Nicaragua': 'Nicaragua','Panam': 'Panama','Panama': 'Panama','Peru': 'Peru','Philippines': 'Philippines', \
                                        'PHL': 'Philippines','PH': 'Philippines','Philippines': 'Philippines','Poland': 'Poland','PL': 'Poland','Puerto Rico': 'Puerto Rico',\
                                        'Republica Dominicana': 'Dominican Republic','RO': 'Romania',\
                                        'Romania': 'Romania','Filipinas': 'Philippines','Republica Checca': 'Czech Republic','Spain': 'Spain','Switzerland': 'Switzerland',\
                                        'Trinidad y Tobago': 'Trinidad y Tobago', \
                                        'UAE': 'United Arab Emirates','UK': 'United Kingdom','United Kingdom': 'United Kingdom','United Arab Emirates': 'United Arab Emirates',\
                                        'Switzerland': 'Switzerland',\
                                        'United States of America': 'United States of America','United States of Americaśśś': 'United States of America',\
                                        'Unites States of America': 'United States of America',\
                                        'US': 'United States of America','USA': 'United States of America'})

    ## Delete all not mapped values and not on country list
    ISM_test = ISM_test[ISM_test['AffectedPersonCountry'].isnull() == False]
    ISM_test = ISM_test[ISM_test['AffectedPersonCountry'].isin(country_list)]

    return ISM_test


def getlistofwords_1(str_desc, nd):
    no_name = nd.search('##CUCARAMACARATITIRIFUE##').get('first_name')
    errcount = 0
    shortest_word_list = []
    cleanstring = str_desc.replace("\\N"," ")
    cleanstring = cleanstring.replace(r"\N"," ")
    cleanstring = re.sub('<.*@.*>', ' ',cleanstring)
    cleanstring = re.sub('[-_"&.,\[\]?!¡#<>;:\'=$()*\\\]', ' ',cleanstring)
    
    #word_list = [re.sub('[?&,:;"><\'.()!¡\[\]-_]', ' ', unidecode.unidecode(wd.strip())) for wd in word_list]
    word_list = cleanstring.split(" ")
    word_list = [unidecode.unidecode(wd.strip()) for wd in word_list]
    
    short_word_list = set(word_list) - set(exclusionlist_1())
    shorter_word_list=[wd for wd in short_word_list if len(wd)>2]
    
    for word in shorter_word_list:
        #Evalua cada palabra y si no es de las exlcluidas por default la agrega a la lista de los palabars del grupo
        if word.find('CID') == -1 and word.find('@CEMEX.COM') == -1 and word.find('@EXT.CEMEX.COM') == -1 and word.find('IMAGE') == -1 and word.find('>>;') == -1 \
                    and word.find('GSC') == -1 and word.find('______') == -1 and word.find('---') == -1 and word.find('  ') == -1 and word.find('GIF') == -1 \
                    and word.find('SAFELINKS') == -1 and word.find('ATTACHE') == -1 and word.find('ADDRESS') == -1 \
                    and word.find('PNG') == -1 and word.find('E0') == -1 and word.find('E-') == -1 and word.find('****') == -1 and word.find('#') == -1 and word.find('LISTENER') \
                    and word.find('YOU') == -1 and word.find('@') == -1 and not word.startswith('ATT00') and not word.startswith('/')\
                    and not word.startswith('+') and not word[0].isdigit() and not word[1].isdigit() and not word.startswith('<') \
                    and not word.startswith('/>') and not word.startswith('CR1') and not word.startswith('CR2') and not word.startswith('CR3') and not word.startswith('CR4') \
                    and not word.startswith('CR5') and not word.startswith('CR6') and not word.startswith('CR7') and not word.startswith('CR8') and not word.startswith('CR9') \
                    and not word.startswith('@') and not word.startswith('?') and not word.startswith('"') and not word.startswith('|'):
                
            try:
                is_name = nd.search(word).get('first_name').get('country').get('Mexico')
                if is_name == no_name:
                    shortest_word_list.append(word)
            except:
                    errcount = errcount + 1
                    shortest_word_list.append(word)
    
    return shortest_word_list

def exclusionlist_1():
        
    exclusionlist = ['OCTOBER','OCTUBRE','NOVEMBER','NOVIEMBRE','MAY','TUESDAY','MONDAY','WEDNESDAY','LUNES','VIERNES','FRIDAY','JUEVES','0','MARTES',\
        'THURSDAY','DOMINGO','MIERCOLES','JULIO','NOV','OCT','MON','SEPTIEMBRE','SABADO','SEPTEMBER','JAN','DOM','SATURDAY','AGO','MAR','SAB','ABRIL',\
        'FEBRERO','JUNE','SUNDAY','MAYO','AUGUST','AGOSTO','DICIEMBRE','JULY','DECEMBER','THU','APRIL','JUE','FRI','MARZO','SEP','AUG','LUN','JANUARY','JUNIO',\
        'ENERO','TUE','MIE','SAT','JUL','JUN','APR','FEBRUARY','DIC','WED','SUN','MARCH','VIE','FEB',\
        'SCREENSHOT','THE','AND','ATTACHMENT','PLEASE','HELLO','FOR','CEMEX','SUBJECT','FAVOR','RE','CAUTION',\
        'REGARDS','SALUDOS','listener','FROM','SUBJECT','CC','TO','FW','0','BUT','DIA','APOYO','SENT','WITH','THAT','COMO','CAN',\
        'HELP','BUT','LES','SUS', 'ADJUNT','SENDER','RECEIVED', \
        'SENT:','GSC', 'THIS', 'DEL','POR','PARA','QUE','CON','WAS','ARE','NOT','THANKS','THANK','LAS','ESTE','E-MAIL','SAVED','LOS','LAS','ESTE', \
        'BUEN','BUENOS','QUEDO','BEST','ASUNTO','DEAR','GRACIAS','GRACIAS','UNA','SALUDOS','RESOLVED','ATENTA','CORDIALES',\
        'ADJUNTO','ADJUNTA','ADJUNTOS','ADJUNTAMOS','ADJUNTANDO','*ADJUNTO','ATTACHED','ATTACHMENTS','ATTACH','ATTACHING','ATTACHED','ATTACHEE', \
        'ADDRESS','DIRECCION',\
        'A','ANTE','ANTES','BAJO','CON','CONTRA','DE','DESDE','PARA','POR','SEGUN','SIN','SOBRE','TRAS','PERO','SAN','NOS','NAME','PUEDE','PUEDA', \
        'UN','UNOS','UNA','UNAS','UNO','MAS','ETC','DIV', 'DUDA', 'SON' \
        'HOLA','BUEN','BUENAS','DIA','BUENOS','BUENA','DIAS','GRACIAS','MUCHAS','TARDES','SIGUIENTE','SIGUIENTES', 'PODRIAN',\
         'ESTA','ESTO','ESTAS','ESTOS','ESA','ESOS','ESAS','ESOS',\
         'AYUDA','TIENE','S/N','DEBE','RESPUESTA', 'TAMBIEN','SIGUE','FUE',\
         'HTTP','HTTPS','WWW','COM','MAILTO','COM/','JPG','CEL','NAME','TEL','TELEFONO',\
        'MAS', 'SOLICITUD','ATTENTION','REQUEST','TEAM','SERVICIO','TODOS', 'NOMBRE','N/A']

    return exclusionlist

def buildwordsindex_1(ISM_wgwords_all, delete_limit, max_word_size):
    #Eliminar palabras que nod dan valor por pocas repeticiones.
    minimum_repeat = delete_limit
    CountedWords = pd.DataFrame(ISM_wgwords_all.groupby(['WORDS'])[['WORDS']].count())
    CountedWords = CountedWords.drop(CountedWords[CountedWords['WORDS'] < minimum_repeat].index)
    CountedWords['WORDS'] = CountedWords.index
    CountedWords.rename(columns ={'WORDS':'VALUES'}, inplace = True)

    ## Hacemos Inner Merge con ISM_wgwords para dejar solo las palabras útiles
    ISM_wgwords_all = pd.merge(ISM_wgwords_all, CountedWords, on="WORDS")
    ISM_wgwords_all = ISM_wgwords_all.drop(ISM_wgwords_all[ISM_wgwords_all['WORDS'].str.len()>25].index)
    
    ## Creamos columna Unique para identificar combinaciones únicas
    ISM_wgwords_all['UNIQUE'] = ISM_wgwords_all['COUNTRY'] + '<<|>>' + ISM_wgwords_all['OWNERGROUP']+ '<<|>>' + ISM_wgwords_all['WORDS']
    arr_wgwords_uniques = ISM_wgwords_all['UNIQUE'].unique()

    #Convertimos en dataframe y separamos las columnas
    df_uniquewords = pd.DataFrame(arr_wgwords_uniques, columns=['UNIQUE'])
    df_uniquewords[['COUNTRY','X1', 'OWNERGROUP', 'X2', 'WORDS']] = df_uniquewords['UNIQUE'].str.split('<<|>>', 5, expand=True)
    df_uniquewords = df_uniquewords[['COUNTRY', 'OWNERGROUP', 'WORDS']]

    #Guardamos los resultados en in dataframe = ISM_wgwords_uniques
    ISM_wgwords_uniques = df_uniquewords
    
    #Movemos los valores al índice para acelerar las búsquedas en la preparadación del trainin data sert
    ISM_wgwords_uniques['STR_INDEX']=ISM_wgwords_uniques['COUNTRY']+"-"+ISM_wgwords_uniques['OWNERGROUP']
    ISM_wgwords_uniques.index = ISM_wgwords_uniques['STR_INDEX']
    return ISM_wgwords_uniques

def training_dataset_1(ISM_test, TrainingData, ISM_wgwords_uniques, square_values):
## ISM_Test datos de ISM Country / Ownergroups / Full Text
## TrainingData Estructura de Salida
## ISM_wgwords_uniques: Calisficador de Palabras: Country / Ownergroup / Words
## square_values: Indica si se elevan al cuadrado los valores de matches de palabras (Bolean)
    nd = NameDataset()
    GroupColumnsNames =ISM_wgwords_uniques['OWNERGROUP'].unique()
    errcount = 0
    count = 0
    count2 = 0
    count_check = 1
    
    #ISM_test['FULLDESC_WORDS'] = getlistofwords_1(ISM_test['FULLDESC'], nd) -- Opcion pa optimizar
    
    for numindex, ind in enumerate(ISM_test.index):
        count = count + 1
        if count == count_check:
            print(datetime.now())
            print(count)

            if count == count_check:
                count_check = count_check + count_check

        word_list = getlistofwords_1(ISM_test['FULLDESC'][ind], nd)        
        #word_list = ISM_test['FULLDESC_WORDS'][ind] --> Opcion pa optimizar
        
        TrainingData = TrainingData.append({'TICKETID':ISM_test['TICKETID'][ind], 'COUNTRY':ISM_test['AffectedPersonCountry'][ind],'OWNERGROUP':ISM_test['OWNERGROUP'][ind]}, ignore_index=True) 
        #TrainingData = TrainingData.append([ISM_test['AffectedPersonCountry'][ind],ISM_test['OWNERGROUP'][ind]], ['COUNTRY','OWNERGROUP'],[numindex])
        
        ## Seteo valores de columnas de grupos del registros nuevo para que no sean null y puedan sumarse
        for column in TrainingData.columns:
            if column not in [0,'COUNTRY','OWNERGROUP','TICKETID']:
                TrainingData[column][numindex] = 0

        for columns in GroupColumnsNames:
            #grouplistwords = ISM_wgwords_uniques[(ISM_wgwords_uniques['COUNTRY']==ISM_test['AffectedPersonCountry'][ind]) &(ISM_wgwords_uniques['OWNERGROUP']==columns) ][['WORDS']]
            searchindex = ISM_test['AffectedPersonCountry'][ind]+"-"+columns
            
            try:
                grouplistwords = ISM_wgwords_uniques.loc[searchindex]
                grouplistwords = grouplistwords['WORDS'].to_list()

                list_matches = set(word_list) & set(grouplistwords)
                if len(list_matches) > 0:
                        #TrainingData[listwords['OWNERGROUP'][listidx]][numindex] = TrainingData[listwords['OWNERGROUP'][listidx]][numindex] + len(list_matches)
                    if square_values:
                        TrainingData[columns][numindex] = TrainingData[columns][numindex] + (len(list_matches))**2
                    else:
                        TrainingData[columns][numindex] = TrainingData[columns][numindex] + len(list_matches)
            except:
                errcount = errcount + 1
            else:
                errcount = errcount + 1
            
            
    print('Finished: ', datetime.now(), ' Errors:',errcount)
    return TrainingData



In [103]:
ISM_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14450 entries, 16 to 428978
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   TICKETID               14450 non-null  object
 1   FULLDESC               14450 non-null  object
 2   AffectedPersonCountry  14450 non-null  object
 3   OWNERGROUP             14450 non-null  object
dtypes: object(4)
memory usage: 564.5+ KB


In [13]:
ISM_wgwords = pd.DataFrame(columns=['COUNTRY', 'OWNERGROUP', 'WORDS'])
ISM_wgwords_1 = PrepareWordLists_1(datasets1, ISM_wgwords)

2022-03-30 23:26:40.579872
1
2022-03-30 23:26:48.277541
501
2022-03-30 23:26:56.596465
1001
2022-03-30 23:27:06.120930
1501
2022-03-30 23:27:15.571395
2001
2022-03-30 23:27:27.328206
2501
2022-03-30 23:27:39.521371
3001
2022-03-30 23:27:53.788046
3501
2022-03-30 23:28:08.948961
4001
2022-03-30 23:28:24.677183
4501
2022-03-30 23:28:43.929754
5001
2022-03-30 23:29:02.087353
5501
2022-03-30 23:29:24.815312
6001
2022-03-30 23:29:49.127605
6501
2022-03-30 23:30:14.083940
7001
2022-03-30 23:30:44.207029
7501
2022-03-30 23:31:16.813487
8001
2022-03-30 23:31:48.543090
8501
2022-03-30 23:32:20.749822
9001
2022-03-30 23:32:55.771523
9501
2022-03-30 23:33:33.310627
Words:  18467  Groups:  145  Countries: 1


In [17]:
ISM_wgwords_2 = PrepareWordLists_1(datasets2, ISM_wgwords)

2022-03-30 23:55:58.771014
1
2022-03-30 23:56:09.039094
501
2022-03-30 23:56:19.001120
1001
2022-03-30 23:56:29.380014
1501
2022-03-30 23:56:43.790259
2001
2022-03-30 23:56:56.462840
2501
2022-03-30 23:57:11.014562
3001
2022-03-30 23:57:27.679196
3501
2022-03-30 23:57:44.385665
4001
2022-03-30 23:58:04.365508
4501
2022-03-30 23:58:23.399158
5001
2022-03-30 23:58:45.670828
5501
2022-03-30 23:59:10.740776
6001
2022-03-30 23:59:38.222512
6501
2022-03-31 00:00:09.264088
7001
2022-03-31 00:00:39.893188
7501
2022-03-31 00:01:16.647223
8001
2022-03-31 00:01:55.418736
8501
2022-03-31 00:02:33.735424
9001
2022-03-31 00:03:12.754647
9501
2022-03-31 00:03:50.661367
Words:  19446  Groups:  150  Countries: 1


In [18]:
ISM_wgwords_3 = PrepareWordLists_1(datasets3, ISM_wgwords)

2022-03-31 00:04:47.973305
1
2022-03-31 00:04:56.969864
501
2022-03-31 00:05:06.683753
1001
2022-03-31 00:05:16.683218
1501
2022-03-31 00:05:26.344198
2001
2022-03-31 00:05:36.914048
2501
2022-03-31 00:05:49.819184
3001
2022-03-31 00:06:05.166966
3501
2022-03-31 00:06:21.680038
4001
2022-03-31 00:06:39.416454
4501
2022-03-31 00:06:57.909615
5001
2022-03-31 00:07:19.020735
5501
2022-03-31 00:07:43.635407
6001
2022-03-31 00:08:07.488742
6501
2022-03-31 00:08:35.845512
7001
2022-03-31 00:09:03.788518
7501
2022-03-31 00:09:39.785661
8001
2022-03-31 00:10:16.418027
8501
2022-03-31 00:10:55.539583
9001
2022-03-31 00:11:36.893420
9501
2022-03-31 00:12:19.429744
Words:  19263  Groups:  143  Countries: 1


In [19]:
ISM_wgwords_4 = PrepareWordLists_1(datasets4, ISM_wgwords)

2022-03-31 00:13:12.633362
1
2022-03-31 00:13:20.975129
501
2022-03-31 00:13:30.510755
1001
2022-03-31 00:13:39.722977
1501
2022-03-31 00:13:50.831448
2001
2022-03-31 00:14:01.939491
2501
2022-03-31 00:14:16.102017
3001
2022-03-31 00:14:30.981321
3501
2022-03-31 00:14:46.877543
4001
2022-03-31 00:15:05.388514
4501
2022-03-31 00:15:25.312055
5001
2022-03-31 00:15:47.606615
5501
2022-03-31 00:16:14.270783
6001
2022-03-31 00:16:39.219978
6501
2022-03-31 00:17:07.743706
7001
2022-03-31 00:17:37.796924
7501
2022-03-31 00:18:10.718125
8001
2022-03-31 00:18:42.321935
8501
2022-03-31 00:19:19.744553
9001
2022-03-31 00:19:57.371718
9501
2022-03-31 00:20:38.568168
Words:  19137  Groups:  146  Countries: 1


In [20]:
ISM_wgwords_5 = PrepareWordLists_1(datasets5, ISM_wgwords)

2022-03-31 00:20:49.060520
1
2022-03-31 00:20:58.090125
501
2022-03-31 00:21:07.927525
1001
2022-03-31 00:21:17.860374
1501
2022-03-31 00:21:28.607529
2001
2022-03-31 00:21:40.114929
2501
2022-03-31 00:21:52.714069
3001
2022-03-31 00:22:08.972853
3501
2022-03-31 00:22:26.345446
4001
2022-03-31 00:22:42.125223
4501
2022-03-31 00:22:45.851068
Words:  11950  Groups:  120  Countries: 1


In [110]:
ISM_wgwords_all = ISM_wgwords_1.append(ISM_wgwords_2)
ISM_wgwords_all = ISM_wgwords_all.append(ISM_wgwords_3)
ISM_wgwords_all = ISM_wgwords_all.append(ISM_wgwords_4)
ISM_wgwords_all = ISM_wgwords_all.append(ISM_wgwords_5)
ISM_wgwords_uniques = buildwordsindex_1(ISM_wgwords_all, 2, 100)
with open('ISM_wgwords_uniques', 'wb') as f:
    pickle.dump(ISM_wgwords_uniques, f, protocol=pickle.HIGHEST_PROTOCOL)

In [68]:
ISM_wgwords_uniques.to_excel(f"./Data/ISM_wgwords_uniques_shortprocess.xlsx")
ISM_wgwords_uniques

,COUNTRY,OWNERGROUP,WORDS,STR_INDEX
STR_INDEX,,,,
Mexico-I-CEX-MXBOSAP-SEC-APP-L3,Mexico,I-CEX-MXBOSAP-SEC-APP-L3,PROCESSED,Mexico-I-CEX-MXBOSAP-SEC-APP-L3
Mexico-I-CEX-MX-OP-GINCO-GSC-L1,Mexico,I-CEX-MX-OP-GINCO-GSC-L1,PROCESSED,Mexico-I-CEX-MX-OP-GINCO-GSC-L1
Mexico-I-CEX-MX-COM-SAP-SL-APP-L3,Mexico,I-CEX-MX-COM-SAP-SL-APP-L3,PROCESSED,Mexico-I-CEX-MX-COM-SAP-SL-APP-L3
Mexico-I-CEX-MX-COM-LE-GSC-L1,Mexico,I-CEX-MX-COM-LE-GSC-L1,PROCESSED,Mexico-I-CEX-MX-COM-LE-GSC-L1
Mexico-I-CEX-IN-IT-IAM-DTC-L2,Mexico,I-CEX-IN-IT-IAM-DTC-L2,PROCESSED,Mexico-I-CEX-IN-IT-IAM-DTC-L2
...,...,...,...,...
Mexico-I-CEX-MX-COM-LE-GSC-L1,Mexico,I-CEX-MX-COM-LE-GSC-L1,GLOBALS,Mexico-I-CEX-MX-COM-LE-GSC-L1
Mexico-I-CEX-MX-PSD-LE-PA-L3,Mexico,I-CEX-MX-PSD-LE-PA-L3,CORREN,Mexico-I-CEX-MX-PSD-LE-PA-L3
Mexico-I-CEX-EMEAA-SAPSEC-APP-L3,Mexico,I-CEX-EMEAA-SAPSEC-APP-L3,ZAMORA/MEXICO/IBM,Mexico-I-CEX-EMEAA-SAPSEC-APP-L3


In [93]:
TrainingData = trainingstructuredata(ISM_wgwords_uniques)

In [94]:
square_values = True

ISM_Training_1 = training_dataset_1(ISM_test_set1, TrainingData, ISM_wgwords_uniques, square_values)

2022-03-31 12:49:19.006911
1
2022-03-31 12:49:19.761748
2
2022-03-31 12:49:20.741703
4
2022-03-31 12:49:23.864987
8
2022-03-31 12:49:27.840817
16
2022-03-31 12:49:37.456276
32
2022-03-31 12:49:53.576583
64
2022-03-31 12:50:24.288079
128
2022-03-31 12:51:28.766631
256
2022-03-31 12:53:38.636681
512
2022-03-31 12:57:47.775924
1024
2022-03-31 13:06:32.001359
2048
2022-03-31 13:23:47.175260
4096
Finished:  2022-03-31 13:32:00.382998  Errors: 951552


In [70]:
ISM_Training_2 = training_dataset_1(ISM_test_set2, TrainingData, ISM_wgwords_uniques, square_values)

2022-03-31 10:26:34.389107
1
2022-03-31 10:26:34.921158
2
2022-03-31 10:26:35.769558
4
2022-03-31 10:26:37.411086
8
2022-03-31 10:26:40.735512
16
2022-03-31 10:26:47.224009
32
2022-03-31 10:27:00.788506
64
2022-03-31 10:27:26.626981
128
2022-03-31 10:28:21.614526
256
2022-03-31 10:30:10.015273
512
2022-03-31 10:33:45.905502
1024
2022-03-31 10:41:17.444016
2048
2022-03-31 10:56:08.083747
4096
Finished:  2022-03-31 11:03:24.183490  Errors: 959808


In [71]:
ISM_Training_3 = training_dataset_1(ISM_test_set3, TrainingData, ISM_wgwords_uniques, square_values)

2022-03-31 11:04:24.867795
1
2022-03-31 11:04:25.535112
2
2022-03-31 11:04:26.777605
4
2022-03-31 11:04:29.142685
8
2022-03-31 11:04:34.086872
16
2022-03-31 11:04:42.959299
32
2022-03-31 11:05:00.439030
64
2022-03-31 11:05:41.128054
128
2022-03-31 11:07:02.451684
256
2022-03-31 11:09:30.374430
512
2022-03-31 11:14:53.871663
1024
2022-03-31 11:25:53.409197
2048
2022-03-31 11:45:32.012884
4096
Finished:  2022-03-31 11:48:43.038654  Errors: 854208
